# Semantic alignement with OpenAI text-embedding-3-small model

## Step 1 - Build a dataset of native edition chunks

In [86]:
import spacy
nlp_ru = spacy.load("ru_core_news_lg")
def split_into_chunks(text: str, max_chars: int = 300) -> list:
    doc = nlp_ru(text)
    sentences = [sent.text.strip() for sent in doc.sents]

    chunks = []
    current_chunk = ""
    for sent in sentences:
        if len(current_chunk) + len(sent) <= max_chars:
            current_chunk += (" " if current_chunk else "") + sent
        else:
            chunks.append(current_chunk)
            current_chunk = sent
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [87]:
from pathlib import Path
def load_text_from_file(filepath: str) -> str:
    return Path(filepath).read_text(encoding='utf-8')
text_ru = load_text_from_file("output_text_rus.txt")
chunks_ru = split_into_chunks(text_ru.replace("\n", " "), max_chars=300)

In [88]:
from IPython.display import display
display(chunks_ru[:3])

['Много лет спустя, перед самым расстрелом, полковник Аурелиано Буэндия припомнит тот далекий день, когда отец повел его поглядеть на лед.',
 'Макондо был тогда небольшим поселком из двадцати глинобитных, с камышовыми кровлями домишек, стоявших на берегу реки, которая несла свои прозрачные воды по ложу из белых, гладких и огромных, как доисторические яйца, валунов.',
 'Мир был таким первозданным, что многие вещи не имели названия и на них просто тыкали пальцем. Каждый год в марте месяце лохмотное цыганское племя ставило свой шатер близ поселка, и под звонкое дребезжание бубнов и визготню свистулек пришельцы показывали жителям новейшие изобретения.']

In [89]:
len(chunks_ru)

213

## Step 2 - Build a dataset of target edition chunks

In [90]:
import es_core_news_sm
nlp_es = es_core_news_sm.load()

text="""
Lo envió a las autoridades acompañado de numerosos testimonios sobre sus experiencias
y de varios pliegos de dibujos explicativos, al cuidado de un mensajero que atravesó la sierra,
se extravió en pantanos desmesurados, remontó ríos tormentosos y estuvo a punto de perecer bajo el azote de las fieras,
la desesperación y la peste, antes de conseguir una ruta de enlace con las mulas del correo.
"""
doc = nlp_es(text)
doc

Lo envió a las autoridades acompañado de numerosos testimonios sobre sus experiencias y de varios pliegos de dibujos explicativos, al cuidado de un mensajero que atravesó la sierra, se extravió en pantanos desmesurados, remontó ríos tormentosos y estuvo a punto de perecer bajo el azote de las fieras, la desesperación y la peste, antes de conseguir una ruta de enlace con las mulas del correo.

In [158]:
from more_itertools import split_at
def break_long_sentences(doc):
    sublists = list(" ".join(line) for line in split_at([d.text for d in doc], lambda x: x == ","))
    return [chunk + ',' if i < len(sublists) - 1 else chunk for i, chunk in enumerate(sublists)]

In [159]:
break_long_sentences(doc)

['Lo envió a las autoridades acompañado de numerosos testimonios sobre sus experiencias y de varios pliegos de dibujos explicativos,',
 'al cuidado de un mensajero que atravesó la sierra,',
 'se extravió en pantanos desmesurados,',
 'remontó ríos tormentosos y estuvo a punto de perecer bajo el azote de las fieras,',
 'la desesperación y la peste,',
 'antes de conseguir una ruta de enlace con las mulas del correo .']

In [164]:
from itertools import chain
def get_es_chunks(text: str) -> list:
    doc = nlp_es(text)

    chunks = []
    current_chunk = ""
    parts = [break_long_sentences(s) for s in doc.sents]
    return list(chain.from_iterable(parts))

In [161]:
text_es = load_text_from_file("output_text_spa.txt")
chunks_es = get_es_chunks(text_es.replace("\n", " "))

In [162]:
len(chunks_es)

1933

In [163]:
chunks_es[:10]

['Muchos años después,',
 'frente al pelotón de fusilamiento,',
 'el coronel Aureliano Buendía había de recordar aquella tarde remota en que su padre 10 llevó a conocer el hielo .',
 'Macondo era entonces una aldea de veinte casas de barro y cañabrava construidas a la orilla de un río de aguas diáfanas que se precipitaban por un lecho de piedras pulidas,',
 'blancas y enormes como huevos prehistóricos .',
 'EI mundo era tan reciente,',
 'que muchas cosas carecían de nombre,',
 'y para mencionarlas había que señalarlas con el dedo .',
 'Todos los años,',
 'por el mes de marzo,']

## Step 3 - Embed and match

In [124]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI(max_retries=5)

In [126]:
from typing import List
def get_embedding(text: str, model="text-embedding-3-small", **kwargs) -> List[float]:
    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")

    response = client.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding

In [168]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [171]:
display([chunks_ru[0], " ".join(chunks_es[0:3])])

['Много лет спустя, перед самым расстрелом, полковник Аурелиано Буэндия припомнит тот далекий день, когда отец повел его поглядеть на лед.',
 'Muchos años después, frente al pelotón de fusilamiento, el coronel Aureliano Buendía había de recordar aquella tarde remota en que su padre 10 llevó a conocer el hielo .']

In [172]:
cosine_similarity(get_embedding(chunks_ru[0]), get_embedding(" ".join(chunks_es[0:3])))

np.float64(0.5534043118961923)

In [173]:
from googletrans import Translator

translator = Translator()
async def translate_russian_to_spanish(text: str) -> str:
    result = await translator.translate(text, src="ru", dest="es")
    return result.text

In [177]:
cosine_similarity(get_embedding(await translate_russian_to_spanish(chunks_ru[0])), get_embedding(" ".join(chunks_es[0:3])))

np.float64(0.8654237965619576)

In [178]:
cosine_similarity(get_embedding(await translate_russian_to_spanish(chunks_ru[1])), get_embedding(" ".join(chunks_es[3:4])))

np.float64(0.8540677287335801)

In [179]:
cosine_similarity(get_embedding(await translate_russian_to_spanish(chunks_ru[1])), get_embedding(" ".join(chunks_es[3:5])))

np.float64(0.9072993472482045)

In [180]:
cosine_similarity(get_embedding(await translate_russian_to_spanish(chunks_ru[1])), get_embedding(" ".join(chunks_es[3:6])))

np.float64(0.8896446964704324)

In [182]:
display([chunks_ru[1], chunks_es[3:5]])

['Макондо был тогда небольшим поселком из двадцати глинобитных, с камышовыми кровлями домишек, стоявших на берегу реки, которая несла свои прозрачные воды по ложу из белых, гладких и огромных, как доисторические яйца, валунов.',
 ['Macondo era entonces una aldea de veinte casas de barro y cañabrava construidas a la orilla de un río de aguas diáfanas que se precipitaban por un lecho de piedras pulidas,',
  'blancas y enormes como huevos prehistóricos .']]